In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Data Preparation

In [2]:
data = pd.read_csv('databaru.csv')

In [3]:
weather_mapping = {
    'clear': 'Sunny',
    'clouds': 'Sunny',
    'haze': 'Sunny',
    'rain': 'Rain',
    'drizzle': 'Rain',
    'thunderstorm': 'Rain',
    'squall': 'Rain'
}
data['Weather'] = data['Weather'].map(weather_mapping)

In [4]:
label_encoder = LabelEncoder()
data['Weather'] = label_encoder.fit_transform(data['Weather'])

# Weather Classification

In [5]:
X = data['Place_Name'].values
y = data['Weather'].values

In [6]:
y_encoded = label_encoder.fit_transform(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [9]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [10]:
max_seq_length = max(len(seq) for seq in X_train_seq)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_seq_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_seq_length)

# Model

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 16, input_length=max_seq_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
12/12 [==============================] - 2s 6ms/step - loss: 0.6684 - accuracy: 0.9972
Epoch 2/10
12/12 [==============================] - 0s 6ms/step - loss: 0.6328 - accuracy: 1.0000
Epoch 3/10
12/12 [==============================] - 0s 5ms/step - loss: 0.5957 - accuracy: 1.0000
Epoch 4/10
12/12 [==============================] - 0s 5ms/step - loss: 0.5538 - accuracy: 1.0000
Epoch 5/10
12/12 [==============================] - 0s 5ms/step - loss: 0.5056 - accuracy: 1.0000
Epoch 6/10
12/12 [==============================] - 0s 5ms/step - loss: 0.4510 - accuracy: 1.0000
Epoch 7/10
12/12 [==============================] - 0s 4ms/step - loss: 0.3916 - accuracy: 1.0000
Epoch 8/10
12/12 [==============================] - 0s 5ms/step - loss: 0.3303 - accuracy: 1.0000
Epoch 9/10
12/12 [==============================] - 0s 5ms/step - loss: 0.2706 - accuracy: 1.0000
Epoch 10/10
12/12 [==============================] - 0s 5ms/step - loss: 0.2159 - accuracy: 1.0000


In [14]:
city_to_coordinates = {
    'Jakarta Pusat': (-6.186486, 106.834091),
    'Jakarta Barat': (-6.135200, 106.813301),
    'Jakarta Timur': (-6.230702, 106.882744),
    'Jakarta Selatan': (-6.300641, 106.814095)
}

In [15]:
city = input("Enter your city (Jakarta Pusat, Jakarta Barat, Jakarta Timur, Jakarta Selatan): ")

Enter your city (Jakarta Pusat, Jakarta Barat, Jakarta Timur, Jakarta Selatan):  Jakarta Selatan


In [16]:
if city not in city_to_coordinates:
    print("Invalid city!")
else:
    latitude, longitude = city_to_coordinates[city]
    
    url = f'http://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid=6efe5d86568540c60dfc5ba4a75a7bbe'

    response = requests.get(url)
    weather_data = response.json()
    weather_description = weather_data['weather'][0]['main'].lower()

    if weather_description in weather_mapping:
        mapped_weather = weather_mapping[weather_description]
    else:
        print(f"Unknown weather description: {weather_description}")
        exit()
        
    predicted_probabilities = model.predict(np.array([mapped_weather]))
    predicted_label_index = np.argmax(predicted_probabilities)
    predicted_weather = label_encoder.inverse_transform([predicted_label_index])[0]

    recommended_attractions = data[data['Weather'] == predicted_weather]['Place_Name'].unique()
    if len(recommended_attractions) > 0:
        print(f"Recommended attractions in {city} based on the weather ({weather_description}):")
        for attraction in recommended_attractions:
            print(attraction)
    else:
        print(f"No attractions found for the current weather in {city}.")

ValueError: in user code:

    File "C:\Users\ihsan\miniconda3\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\ihsan\miniconda3\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ihsan\miniconda3\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\ihsan\miniconda3\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\ihsan\miniconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ihsan\miniconda3\lib\site-packages\keras\engine\input_spec.py", line 235, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [ ]:
    predicted_probabilities = model.predict(np.array([mapped_weather]))
    predicted_label_index = np.argmax(predicted_probabilities)
    predicted_weather = label_encoder.inverse_transform([predicted_label_index])[0]

    recommended_attractions = data[data['Weather'] == predicted_weather]['Place_Name'].unique()
    if len(recommended_attractions) > 0:
        print(f"Recommended attractions in {city} based on the weather ({weather_description}):")
        for attraction in recommended_attractions:
            print(attraction)
    else:
        print(f"No attractions found for the current weather in {city}.")

In [ ]:
print(latitude)

In [ ]:
print(response.status_code)

In [38]:
print(weather_data)

{'coord': {'lon': 106.8327, 'lat': -6.1864}, 'weather': [{'id': 721, 'main': 'Haze', 'description': 'haze', 'icon': '50n'}, {'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 300, 'feels_like': 302.73, 'temp_min': 298.68, 'temp_max': 301.01, 'pressure': 1012, 'humidity': 82}, 'visibility': 5000, 'wind': {'speed': 1.54, 'deg': 240}, 'rain': {'1h': 0.87}, 'clouds': {'all': 40}, 'dt': 1685460448, 'sys': {'type': 1, 'id': 9383, 'country': 'ID', 'sunrise': 1685400994, 'sunset': 1685443440}, 'timezone': 25200, 'id': 1631845, 'name': 'Pecenongan', 'cod': 200}


In [39]:
print(weather_description)

haze


In [40]:
print(mapped_weather)

Sunny
